In [16]:
# % pylab inline
from pylab import *

In [17]:
# imports
import os
import cv2
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [24]:
def get_paths(dir_path):
    """Receives dir_path, returns 2-dimensional array of certain sizes"""
    image_paths = []
    
    for i, class_name in enumerate(os.listdir(dir_path)):
        class_image_paths = []
        for j, image_name in enumerate(os.listdir(dir_path+class_name)):
            class_image_paths.append(image_name)
        image_paths.append(class_image_paths)
    
    return image_paths

In [28]:
def read_images(dir_path, image_paths):
    
    image_paths = []
    
    for i, class_name in enumerate(os.listdir(dir_path)):
        class_image_paths = []
        for j, image_name in enumerate(os.listdir(dir_path+class_name)):
            class_image_paths.append(image_name)
        image_paths.append(class_image_paths)
    
    X = []
    y = []
    
    class_num = 0
    print(len(image_paths))
    for each_class in enumerate(image_paths):
        for each_image_path in each_class:
#             print(each_image_path)
            image = cv2.imread(each_image_path)
            X.append(image)
            y.append(class_num)
        class_num += 1
    
    X = np.array(X)
    y = np.array(y)
    
    return X, y

In [32]:
train_dir_path1 = 'hw2 data/data1/train/'
train_dir_path2 = 'hw2 data/data2/train/'

print(os.listdir(train_dir_path1))
print(os.listdir(train_dir_path2))

image_paths1 = get_paths(train_dir_path1)
image_paths2 = get_paths(train_dir_path2)

X1, y1 = read_images(train_dir_path1, image_paths1)
X2, y2 = read_images(train_dir_path2, image_paths2)

['FROG', 'HORSE', 'DOG', 'BIRD', 'AIRPLANE']
['Lemon', 'Grape', 'Kiwi', 'Apple Red 1', 'Strawberry']
5
5


In [ ]:
def load_data(dir_path1, dir_path2):
    image_paths1 = get_paths(dir_path1)
    image_paths2 = get_paths(dir_path2)
    
    X1, y1 = read_images(image_paths1)
    X2, y2 = read_images(image_paths2)
    
    X2_resized = np.zeros((X2.shape[0], 32, 32, X2.shape[3]), dtype=np.uint8)
    
    for i in range(X2.shape[0]):
        X2_resized[i,:,:,0] = cv2.resize(X2[i,:,:,0], (32,32))
        X2_resized[i,:,:,1] = cv2.resize(X2[i,:,:,1], (32,32))
        X2_resized[i,:,:,2] = cv2.resize(X2[i,:,:,2], (32,32))
    
    X = np.append(X1, X2_resized, axis=0)
    y = np.append(y1, y2, axis=0)
    
    return X, y

In [ ]:
train_dir_path1 = 'hw2 data/data1/train/'
train_dir_path2 = 'hw2 data/data2/train/'
X_train, y_train= load_data(train_dir_path1, train_dir_path2)

In [ ]:
class CIFAR10(torch.utils.data.dataset.Dataset):
    __Xs = None
    __ys = None
    
    def __init__(self, dir_path1, dir_path2, transform=None):
        self.transform = transform
        self.__Xs, self.__ys = load_data(dir_path1, dir_path2)
        
    def __getitem__(self, index):
        img = self.__Xs[index]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            img = self.transform(img)
            
        # Convert image and label to torch tensors
        img = torch.from_numpy(np.asarray(img))
        label = torch.from_numpy(np.asarray(self.__ys[index]))
        
        return img, label
    
    def __len__(self):
        return self.__Xs.shape[0]

In [ ]:
train_dir_path1 = 'hw2 data/data1/train/'
train_dir_path2 = 'hw2 data/data2/train/'
batch_size = 1

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = CIFAR10(train_dir_path1, train_dir_path2, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
device
torch.cuda.empty_cache()

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
# net.to(device)
net = net.cuda()

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [13]:
epochs = 10

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
#         inputs.to(device)
#         labels.to(device)
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print('epoch %d/%d \t%5d\t loss: %.3f' %
              (epoch + 1, epochs, i + 1, running_loss))
    running_loss = 0.0

print('Finished Training')

epoch 1/10 	 4537	 loss: 5717.593
epoch 2/10 	 4537	 loss: 3453.419
epoch 3/10 	 4537	 loss: 3094.706
epoch 4/10 	 4537	 loss: 2770.625
epoch 5/10 	 4537	 loss: 2450.777
epoch 6/10 	 4537	 loss: 2221.596
epoch 7/10 	 4537	 loss: 2047.247
epoch 8/10 	 4537	 loss: 1824.267
epoch 9/10 	 4537	 loss: 1588.315
epoch 10/10 	 4537	 loss: 1481.285
Finished Training


In [15]:
test_dir_path1 = 'hw2 data/data1/test/'
test_dir_path2 = 'hw2 data/data2/test/'
batch_size = 1

testset = CIFAR10(test_dir_path1, test_dir_path2, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=1)

NotADirectoryError: [Errno 20] Not a directory: 'hw2 data/data1/test/23360_dog.png'